## Task 1 - SQL

### Build SparkSession:

In [3]:
#!pip install findspark
import findspark
#findspark.init()
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkSQL').enableHiveSupport().getOrCreate()

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 58.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=1ae306c19e1968fef80de17e0f259270fb4fc78bd2043057e326d425c153a8a6
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


### Read the json file:

In [4]:
df = spark.read.json('DataFrames_sample.json')

### Display the schema:


In [5]:
df.printSchema()

root
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- HDD: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- W: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Year: long (nullable = true)



### Get all the data when "Model" equal "MacBook Pro":




In [7]:
df.describe().show()

+-------+------------------+-----------------+---------+------------------+-------+----+----------+------------------+-----------------+------------------+
|summary|                 D|                H|      HDD|                Id|  Model| RAM|ScreenSize|                 W|           Weight|              Year|
+-------+------------------+-----------------+---------+------------------+-------+----+----------+------------------+-----------------+------------------+
|  count|                 4|                4|        4|                 4|      4|   4|         4|                 4|                4|                 4|
|   mean|              8.54|           5.5275|     null|               2.5|   null|null|      null|15.797500000000001|           7.4525|            2016.0|
| stddev|0.8114185110040316|9.848551077865888|     null|1.2909944487358056|   null|null|      null| 6.630738395281983|8.935395439859764|0.8164965809276952|
|    min|              7.74|             0.52|128GB SSD|        

### Create TempView:

In [8]:
df.createOrReplaceTempView("sample")

### Display "RAM"column and count "RAM" column:

In [9]:
df.groupby('RAM').count().show()

+----+-----+
| RAM|count|
+----+-----+
|64GB|    1|
|16GB|    1|
| 8GB|    2|
+----+-----+



### Get all columns when "Year" column equal "2015"  

In [10]:
spark.sql("""SELECT *
          FROM sample 
          WHERE Year = 2015""").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all when "Model" start with "M":

In [11]:
spark.sql("""SELECT *
          FROM sample 
          WHERE Model like 'M%' """).show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data when "Model" column equal "MacBook Pro"

In [12]:
spark.sql("""SELECT *
          FROM sample 
          WHERE Model = 'MacBook Pro' """).show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data with Multiple Conditions when "RAM" column equal "8GB" and "Model" column is "Macbook".

In [13]:
spark.sql("""SELECT *
          FROM sample 
          WHERE RAM = '8GB' AND Model = 'MacBook' """).show()

+----+----+---------+---+-------+---+----------+-----+------+----+
|   D|   H|      HDD| Id|  Model|RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-------+---+----------+-----+------+----+
|7.74|0.52|256GB SSD|  2|MacBook|8GB|       12"|11.04|  2.03|2016|
+----+----+---------+---+-------+---+----------+-----+------+----+



### Get all data with Multiple Conditions when "D" greater than or equal "8" and "Model" column is "iMac".

In [14]:
spark.sql("""SELECT *
          FROM sample 
          WHERE D >= 8 AND Model = 'iMac' """).show()

+---+----+-------+---+-----+----+----------+----+------+----+
|  D|   H|    HDD| Id|Model| RAM|ScreenSize|   W|Weight|Year|
+---+----+-------+---+-----+----+----------+----+------+----+
|8.0|20.3|1TB SSD|  4| iMac|64GB|       27"|25.6|  20.8|2017|
+---+----+-------+---+-----+----+----------+----+------+----+



## Task 2


### Read "test1" dataset:

In [15]:
test1= airline_data = 'test1.csv'
df_schema = "Name STRING, age INT, Experience INT, Salary INT"

df1=(spark.read.format('csv')
    .schema(df_schema)
    .option('header','true')
    .load(test1)
)
# register the DataFrame as a temporary view
df1.createOrReplaceTempView("test")

### Display Salary of the people less than or equal to 20000

In [16]:
spark.sql("""SELECT Salary
          FROM test
          WHERE Salary <= 20000 """).show()

+------+
|Salary|
+------+
| 20000|
| 20000|
| 15000|
| 18000|
+------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [17]:
spark.sql("""SELECT Salary
          FROM test
          WHERE Salary <= 20000 AND Salary >= 15000""").show()

+------+
|Salary|
+------+
| 20000|
| 20000|
| 15000|
| 18000|
+------+



## Task 3 

### Read "test3" dataset:

In [18]:
test3= airline_data = 'test3.csv'
df_schema3 = "Name STRING, Department STRING, salary INT"

df3=(spark.read.format('csv')
    .schema(df_schema3)
    .option('header','true')
    .load(test3)
)
# register the DataFrame as a temporary view
df3.createOrReplaceTempView("NewTest")

### Display dataset

In [19]:
df3.show(10)

+---------+------------+------+
|     Name|  Department|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



### Display schema

In [20]:
df3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- salary: integer (nullable = true)



### Group by "Name" column and using sum function on "Name" column

In [21]:
df3.groupby('Name').sum().show()

+---------+-----------+
|     Name|sum(salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



### Group by "Name" column and using avg function on "Name" column

In [22]:
df3.groupby('Name').avg().show()

+---------+------------------+
|     Name|       avg(salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



### Group by "Departments" column and using sum function on "Departments" column

In [24]:
df3.groupby('Department').sum().show()

+------------+-----------+
|  Department|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



### Group by "Departments" column and using mean function on "Departments" column:

In [27]:
df3.groupby('Department').mean().show()

+------------+-----------+
|  Department|avg(salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



Group by "Departments" column and using count function on "Departments" column:

In [28]:
df3.groupby('Department').count().show()

+------------+-----+
|  Department|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



### Apply agg to using sum function get the total of salaries

In [ ]:

df3.agg(sum("salary")).show()

## Task 4

You've been flown to their headquarters in Ulsan, South Korea, to assist them in accurately estimating the number of crew members a ship will need.


They're currently building new ships for certain customers, and they'd like you to create a model and utilize it to estimate how many crew members the ships will require.


Metadata:
1. Measurements of ship size 
2. capacity 
3. crew 
4. age for 158 cruise ships.

It is saved in a csv file for you called "ITI_data.csv". our task is to develop a regression model that will assist in predicting the number of crew members required for future ships. The client also indicated that they have found that particular cruise lines will differ in acceptable crew counts, thus this is most likely an important factor to consider when conducting your investigation.

In [39]:
iti=spark.read.csv('./ITI_data.csv',inferSchema=True,header=True)

### OneHotEncoder 


In [40]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
categoricalCols = [field for (field, dataType) in iti.dtypes
                   if dataType == "string"]
indexOutputCols = [x + "_Index" for x in categoricalCols]
oheOutputCols = [x + "_OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)


###Use VectorAssembler to merge all columns into one column:

In [46]:
from pyspark.ml.feature import VectorAssembler

numericCols = [field for (field,dataType) in iti.dtypes
              if ((dataType=='double'))]
assemblerInputs = oheOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

### Create a Linear Regression Model 

In [ ]:
from pyspark.ml.regression import LinearRegression
vectTrainDF = vecAssembler.transform(iti)
lr = LinearRegression(featuresCol='features',labelCol='crew')
lrModel = lr.fit(vecAssembler)

### Creating a Pipeline

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vecAssembler,lr])
pipelineModel = pipeline.fit(iti)

### Model Evaluation

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
regressionEvaluator = RegressionEvaluator(predictionCol='prediction',
                                         labelCol='crew',
                                         metricName='rmse')
